In [ ]:
!pip install -U openai-whisper
!pip install srt
!pip install googletrans==4.0.0rc1

In [ ]:
import os
import srt
import asyncio
from googletrans import Translator
import nest_asyncio
import subprocess
import logging
import time

nest_asyncio.apply()

# ロガーの初期化
logging.basicConfig(level=logging.INFO)

# 翻訳機の初期化
translator = Translator()

def get_srt_files():
    """現在のディレクトリ内のsrtファイルのリストを取得する。"""
    return [f for f in os.listdir('.') if f.endswith('.srt')]

def select_srt_file():
    """ユーザーにsrtファイルを選択させる。"""
    srt_files = get_srt_files()
    if not srt_files:
        print("No .srt files found in the current directory.")
        return None

    print("Available .srt files:")
    for i, file in enumerate(srt_files, 1):
        print(f"{i}. {file}")

    while True:
        try:
            choice = int(input("Enter the number of the file you want to use: "))
            if 1 <= choice <= len(srt_files):
                return srt_files[choice - 1]
            else:
                print("Invalid choice. Please try again.")
        except ValueError:
            print("Invalid input. Please enter a number.")

def google_translate(text, src='en', dest='ja', max_attempts=5, sleep_time=0.2):
    """Google Translate を使用してテキストを翻訳する。"""
    for attempt in range(max_attempts):
        try:
            return translator.translate(text, src=src, dest=dest)
        except Exception as e:
            logging.warning(f"Attempt {attempt+1}/{max_attempts}: {e}. Retrying...")
            if attempt < max_attempts - 1:
                time.sleep(sleep_time)
            else:
                raise Exception(f"翻訳に失敗しました ({max_attempts} 回の試行の後)。")

def read_srt(input_srt_file):
    """SRTファイルを読み込み、字幕データのリストを返す。"""
    try:
        with open(input_srt_file, 'r', encoding='utf-8', errors='ignore') as file:
            srt_data = file.read()
        return list(srt.parse(srt_data))
    except Exception as e:
        print(f"Error reading SRT file: {e}")
        return []

def write_srt(output_srt_file, subtitles):
    """翻訳された字幕データをSRTファイルに書き込む。"""
    try:
        translated_srt_content = srt.compose(subtitles)
        with open(output_srt_file, "w", encoding="utf-8") as f:
            f.write(translated_srt_content)
    except Exception as e:
        print(f"Error writing SRT file: {e}")

async def translate_subtitles(subtitles, batch_size=10, src='en', dest='ja', sleep_time=0.2):
    """字幕データをバッチ処理で翻訳する。"""
    for i in range(0, len(subtitles), batch_size):
        batch = subtitles[i:i + batch_size]
        for j, subtitle in enumerate(batch):
            translated = google_translate(subtitle.content, src, dest)
            subtitles[i + j].content = translated.text
            progress = (i + j + 1) / len(subtitles) * 100
            print(f"Translation progress: {progress:.2f}%")
        time.sleep(sleep_time)
    return subtitles

async def translate_srt(input_srt_file, output_srt_file, batch_size=10):
    """SRTファイルを読み込み、翻訳して、新しいSRTファイルに書き込む。"""
    subtitles = read_srt(input_srt_file)
    if not subtitles:
        print("No subtitles to translate.")
        return
    translated_subtitles = await translate_subtitles(subtitles, batch_size)
    write_srt(output_srt_file, translated_subtitles)

def add_line_breaks(text, max_length=20, max_lines=2):
    """テキストに改行を追加する。"""
    result = ""
    current_line = ""
    lines = []

    for char in text:
        if char == "、" or char == "。":
            current_line += char
            if len(current_line) > max_length or char == "。":
                lines.append(current_line)
                current_line = ""
        else:
            current_line += char

    if current_line:
        lines.append(current_line)

    while len(lines) > max_lines:
        line1 = lines.pop(0)
        line2 = lines.pop(0)
        combined_line = line1 + line2
        lines.insert(0, combined_line)

    return "\n".join(lines)

def add_line_breaks_srt(srt_text):
    """SRT形式を保持して改行を追加する。"""
    lines = srt_text.split("\n")
    result = ""

    for i, line in enumerate(lines):
        if line.strip().isdigit():
            result += line
        elif "-->" in line:
            result += line
        else:
            result += add_line_breaks(line) + "\n"

        if i < len(lines) - 1 and lines[i + 1] != "" and line != "":
            result += "\n"

    return result.strip()

# 実行
input_srt = select_srt_file()
if input_srt:
    output_srt = 'translated_' + input_srt
    video = input_srt.replace(".srt", ".mp4")

    # SRT翻訳の実行
    asyncio.run(translate_srt(input_srt, output_srt, batch_size=10))

    # 改行されたテキストを取得
    with open(output_srt, 'r', encoding='utf-8') as file:
        srt_text = file.read()

    # 改行を追加
    result = add_line_breaks_srt(srt_text)

    # ファイルへの書き込み
    with open("translated_file_result.srt", "w", encoding='utf-8') as file:
        file.write(result)

    # 字幕付き動画の生成
    subprocess.run([
        'ffmpeg',
        '-hwaccel', 'auto', 
        '-i', f"{video}",
        '-vf', "subtitles=translated_file_result.srt:force_style='FontName=Helvetica,FontSize=11'",
        '-c:v', 'h264_amf', 
        '-c:a', 'copy', 
        f'{video.replace(".mp4", "_jp.mp4")}' 
    ])
else:
    print("No file selected. Exiting.")